### 라이브러리

In [29]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib 
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams['font.family'] = "AppleGothic"

from sklearn.ensemble import RandomForestRegressor

from pycaret.regression import *
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error

### 데이터 처리

In [30]:
train = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/train.csv')
test = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/test.csv')
submission = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/sample_submission.csv')

In [31]:
weekdayrankfordinner = {
    '월' : 1,
    '화' : 2,
    '수' : 4,
    '목' : 3,
    '금' : 5
} 

train['요일(석식)'] = train['요일'].map(weekdayrankfordinner)
test['요일(석식)'] = test['요일'].map(weekdayrankfordinner)

In [32]:
train['일자'] = pd.to_datetime(train['일자'])
test['일자'] = pd.to_datetime(test['일자'])

train['년'] = train['일자'].dt.year
train['월'] = train['일자'].dt.month
train['일'] = train['일자'].dt.day
train['주'] = train['일자'].dt.isocalendar().week
train['요일'] = train['일자'].dt.weekday


test['년'] = test['일자'].dt.year
test['월'] = test['일자'].dt.month
test['일'] = test['일자'].dt.day
test['주'] = test['일자'].dt.isocalendar().week
test['요일'] = test['일자'].dt.weekday

train['주'] = train['주'].astype('int') # '주'의 타입이 Uint로 사용이 안되어 int로 변경해줌.
test['주'] = test['주'].astype('int')

In [33]:
drops = ['조식메뉴', '중식메뉴', '석식메뉴']

train = train.drop(drops, axis=1)
test = test.drop(drops, axis=1)

In [34]:
train['식사가능자수'] = (train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수'])
test['식사가능자수'] = (test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수'])

In [35]:
month1 = {
    1:3,
    2:1,
    3:2,
    4:6,
    5:7,
    6:8,
    7:10,
    8:9,
    9:5,
    10:4,
    11:11,
    12:12
}

month2 = {
    1:11,
    2:2,
    3:1,
    4:4,
    5:8,
    6:6,
    7:9,
    8:7,
    9:5,
    10:3,
    11:10,
    12:12
}
train['월(중식)'] = train['월'].map(month1)
test['월(중식)'] = test['월'].map(month1)

train['월(석식)'] = train['월'].map(month2)
test['월(석식)'] = test['월'].map(month2)

train['월(중식)'] = train['월(중식)'].astype('int') # float형이라 int로 변경
test['월(중식)'] = test['월(중식)'].astype('int')
train['월(석식)'] = train['월(석식)'].astype('int')
test['월(석식)'] = test['월(석식)'].astype('int')

In [36]:
train['공휴일전후'] = 0
test['공휴일전후'] = 0

In [28]:
train['공휴일전후'][4] = 1 #2
train['공휴일전후'][17] = 1 #
#train['공휴일전후'][62] = 1
# train['공휴일전후'][67] = 1
# train['공휴일전후'][82] = 1
train['공휴일전후'][131] = 1
# train['공휴일전후'][130] = 1
train['공휴일전후'][152] = 1
train['공휴일전후'][226] = 1 # ?
train['공휴일전후'][221] = 1 # ?
#train['공휴일전후'][222] = 1
train['공휴일전후'][224] = 1 # ?
train['공휴일전후'][225] = 1 # ?
# train['공휴일전후'][244] = 1
train['공휴일전후'][245] = 1
# train['공휴일전후'][267] = 1
train['공휴일전후'][309] = 1
train['공휴일전후'][310] = 2
train['공휴일전후'][311] = 1 # ?
train['공휴일전후'][330] = 1
train['공휴일전후'][379] = 1
train['공휴일전후'][467] = 1 ## 이거 성능별로지않나?
# train['공휴일전후'][469] = 1
train['공휴일전후'][470] = 1
train['공휴일전후'][502] = 2
# train['공휴일전후'][501] = 1
train['공휴일전후'][511] = 1 # 510 아님? 후가들어감
train['공휴일전후'][565] = 1
train['공휴일전후'][623] = 1
train['공휴일전후'][651] = 1
# train['공휴일전후'][650] = 1
train['공휴일전후'][705] = 2
# train['공휴일전후'][707] = 1
train['공휴일전후'][709] = 2
# train['공휴일전후'][733] = 1
# train['공휴일전후'][748] = 1
# train['공휴일전후'][792] = 1
train['공휴일전후'][815] = 2 # 후인데 효과좋긴함
train['공휴일전후'][864] = 1
# train['공휴일전후'][863] = 1
train['공휴일전후'][950] = 1 # ?
train['공휴일전후'][951] = 1
train['공휴일전후'][953] = 2 # ?
train['공휴일전후'][954] = 1 # 후인데 효과좋긴함
train['공휴일전후'][955] = 1
train['공휴일전후'][971] = 1
# train['공휴일전후'][970] = 1
# train['공휴일전후'][1037] = 1
train['공휴일전후'][1038] = 1
train['공휴일전후'][1099] = 1 # ?
train['공휴일전후'][1129] = 2
# train['공휴일전후'][1128] = 1
train['공휴일전후'][1187] = 1
# train['공휴일전후'][1186] = 1

test['공휴일전후'][10] =2
test['공휴일전후'][20] = 1

```
###그냥 변경해보기

train['공휴일전후'][4] = 1 #2
train['공휴일전후'][17] = 1
#train['공휴일전후'][62] = 1
# train['공휴일전후'][67] = 1
# train['공휴일전후'][82] = 1
train['공휴일전후'][131] = 1
# train['공휴일전후'][130] = 1
train['공휴일전후'][152] = 1
train['공휴일전후'][226] = 1 # ?
train['공휴일전후'][221] = 1 # ?
#train['공휴일전후'][222] = 1
train['공휴일전후'][224] = 1 # ?
train['공휴일전후'][225] = 1 # ?
# train['공휴일전후'][244] = 1
train['공휴일전후'][245] = 1
# train['공휴일전후'][267] = 1
train['공휴일전후'][309] = 1
train['공휴일전후'][310] = 2
train['공휴일전후'][311] = 1 # ?
train['공휴일전후'][330] = 1
train['공휴일전후'][379] = 1
###train['공휴일전후'][467] = 1 ## 이거 성능별로지않나?
# train['공휴일전후'][469] = 1
train['공휴일전후'][470] = 1
train['공휴일전후'][502] = 2
# train['공휴일전후'][501] = 1
###train['공휴일전후'][511] = 1 # 510 아님? 후가들어감
train['공휴일전후'][565] = 1
train['공휴일전후'][623] = 1
train['공휴일전후'][651] = 1
# train['공휴일전후'][650] = 1
train['공휴일전후'][705] = 2
# train['공휴일전후'][707] = 1
train['공휴일전후'][709] = 2
# train['공휴일전후'][733] = 1
# train['공휴일전후'][748] = 1
# train['공휴일전후'][792] = 1
train['공휴일전후'][815] = 2 # 후인데 효과좋긴함
train['공휴일전후'][864] = 1
# train['공휴일전후'][863] = 1
train['공휴일전후'][950] = 1 # ?
train['공휴일전후'][951] = 1
train['공휴일전후'][953] = 2 # ?
train['공휴일전후'][954] = 1 # 후인데 효과좋긴함
train['공휴일전후'][955] = 1
train['공휴일전후'][971] = 1
# train['공휴일전후'][970] = 1
# train['공휴일전후'][1037] = 1
train['공휴일전후'][1038] = 1
train['공휴일전후'][1099] = 1 # ?
train['공휴일전후'][1129] = 2
# train['공휴일전후'][1128] = 1
train['공휴일전후'][1187] = 1
# train['공휴일전후'][1186] = 1

test['공휴일전후'][10] =2
test['공휴일전후'][20] = 1
```

In [38]:
# 원핫인코딩
train = pd.get_dummies(train, columns=['공휴일전후'])
test = pd.get_dummies(test, columns=['공휴일전후'])


In [39]:
test['공휴일전후_0'][20] = 1
test['공휴일전후_1'][20] = 0

In [40]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '중식계', '석식계', '요일(석식)', '년', '월', '일', '주', '식사가능자수',
       '월(중식)', '월(석식)', '공휴일전후_0', '공휴일전후_1', '공휴일전후_2'],
      dtype='object')

In [41]:
train[train.석식계>0]['석식계'].mean() # 0이아닌값 평균

# 현재 1월만 대체
train['석식계'][244] = 478.8605851979346

#train['석식계'][281] = 398
train['석식계'][492] = 478.8605851979346

#train['석식계'][502] = 398
#train['석식계'][510] = 398
#train['석식계'][529] = 398
train['석식계'][730] = 478.8605851979346

#train['석식계'][747] = 398
#train['석식계'][766] = 398
train['석식계'][973] = 478.8605851979346

#train['석식계'][993] = 398

# 1 2 3 월 자기개발의날 대체.

### 최종제출 ( 자기개발1대체,금)

In [42]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor()

In [43]:
# train = train[train.석식계!= 0]

x1_train = train[['요일', '본사시간외근무명령서승인건수', '식사가능자수','월(중식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']] #중식계
x2_train = train[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수','월(석식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']] #석식계

y1_train = train['중식계']
y2_train = train['석식계']

x1_test = test[['요일', '본사시간외근무명령서승인건수', '식사가능자수', '월(중식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']]
x2_test = test[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수', '월(석식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']]

In [44]:
model1 = cat
model2 = cat

model1.fit(x1_train, y1_train) #중식계
pred1 = model1.predict(x1_test)

model2.fit(x2_train, y2_train) #석식계
pred2 = model2.predict(x2_test)

submission['중식계'] = pred1
submission['석식계'] = pred2

Learning rate set to 0.040347
0:	learn: 203.7664601	total: 4.52ms	remaining: 4.52s
1:	learn: 198.8405842	total: 8.03ms	remaining: 4s
2:	learn: 193.8437274	total: 11.2ms	remaining: 3.72s
3:	learn: 189.4049298	total: 13.6ms	remaining: 3.39s
4:	learn: 185.2634891	total: 18.5ms	remaining: 3.69s
5:	learn: 181.0569205	total: 21.8ms	remaining: 3.61s
6:	learn: 176.9920539	total: 25.4ms	remaining: 3.6s
7:	learn: 173.0342559	total: 28.7ms	remaining: 3.55s
8:	learn: 169.2850959	total: 32.1ms	remaining: 3.53s
9:	learn: 165.7586357	total: 34.7ms	remaining: 3.44s
10:	learn: 162.3017274	total: 36.7ms	remaining: 3.3s
11:	learn: 159.0624012	total: 39.5ms	remaining: 3.25s
12:	learn: 156.1831056	total: 42.2ms	remaining: 3.2s
13:	learn: 153.7090549	total: 44.8ms	remaining: 3.16s
14:	learn: 150.8052988	total: 47.4ms	remaining: 3.11s
15:	learn: 148.0971339	total: 49.5ms	remaining: 3.05s
16:	learn: 145.6446120	total: 52.4ms	remaining: 3.03s
17:	learn: 143.4301465	total: 55.1ms	remaining: 3.01s
18:	learn: 141

In [45]:
submission.head(3)

,일자,중식계,석식계
0,2021-01-27,1033.617085,316.346032
1,2021-01-28,976.655747,502.922384
2,2021-01-29,593.057831,237.981969


In [46]:
submission

,일자,중식계,석식계
0,2021-01-27,1033.617085,316.346032
1,2021-01-28,976.655747,502.922384
2,2021-01-29,593.057831,237.981969
3,2021-02-01,1210.781725,520.118838
4,2021-02-02,1048.526036,567.822235
5,2021-02-03,1078.981988,509.855142
6,2021-02-04,994.953620,562.610231
7,2021-02-05,729.143635,333.453968
8,2021-02-08,1288.809661,681.031673
9,2021-02-09,1094.035577,593.152713


In [18]:
submission['석식계'].mean()

517.5148429681514

In [19]:
submission.to_csv('자기개발1대체,금.csv', index=False)

In [20]:
train['식사가능자수']

0       2551.0
1       2551.0
2       2545.0
3       2497.0
4       2323.0
         ...  
1200    2517.0
1201    2540.0
1202    2425.0
1203    2549.0
1204    2552.0
Name: 식사가능자수, Length: 1205, dtype: float64

In [22]:
train.groupby('요일').mean()['석식계']

요일
0    539.533333
1    520.129167
2    371.629466
3    480.401639
4    404.938017
Name: 석식계, dtype: float64

In [23]:
train.query('석식계 == 0 & 월 <4')['석식계']

262    0.0
281    0.0
502    0.0
510    0.0
529    0.0
747    0.0
766    0.0
993    0.0
Name: 석식계, dtype: float64